# Simple watsonx.ai Environment (Binder-ready)

This notebook is a Binder-friendly version of your **simple watsonx environment**.

It is designed to run in a Jupyter Notebook environment launched via
[Binder](https://mybinder.org), but it will also work in any standard
Jupyter environment (local, JupyterHub, etc.).

## What this notebook does

1. Installs the required Python libraries (`ibm-watsonx-ai`, `langchain-ibm`, `python-dotenv`).
2. Asks you for your IBM Cloud / watsonx.ai credentials (API key, project id, URL).
3. Runs a **quickstart** example using the native `ibm_watsonx_ai` SDK.
4. Optionally shows how to call watsonx.ai through **LangChain**.

> ⚠️ **Security note**
>
> - Do **not** commit API keys into a public Git repository.
> - When using Binder on a public repo, always enter your keys **at runtime** using the prompts in this notebook.
> - Any `.env` file in a public repo is visible to everyone.


## How to launch this notebook on Binder

1. Push this notebook (for example as `simple_watsonx_environment_binder.ipynb`) to a Git repository (e.g. on GitHub or GitLab).
2. In the same repository, create a `requirements.txt` file with at least:

   ```text
   ibm-watsonx-ai
   langchain-ibm
   python-dotenv
   ```

3. Go to [https://mybinder.org](https://mybinder.org).
4. Enter your repository URL and, in the **File to open** field, put the path to this notebook
   (e.g. `simple_watsonx_environment_binder.ipynb`).
5. Click **Launch**. Binder will build an image with your dependencies and then open Jupyter.
6. In Jupyter, open this notebook (if it is not already opened) and run the cells from top to bottom.

If you don't want to use `requirements.txt`, this notebook also contains a cell that installs
the required libraries via `pip` at runtime.

In [ ]:
# 1️⃣ Install dependencies
# If you're using Binder with a requirements.txt that already includes these
# libraries, you can skip this cell. Otherwise, run it to install them here.

!pip install -q ibm-watsonx-ai langchain-ibm python-dotenv
print("✅ Libraries installed (or already present).")

In [ ]:
# 2️⃣ Configure your IBM Cloud / watsonx.ai credentials
# These values live only in the current notebook kernel.

import os
import getpass

print("⚠️ Your keys are kept only in this running environment. Do NOT commit them to Git.")

# Required
IBM_CLOUD_API_KEY = getpass.getpass("IBM Cloud API key: ")
IBM_CLOUD_PROJECT_ID = input("IBM Cloud project_id: ").strip()

# Region URL – change if you use a different region (for example: eu-de, ca-tor, etc.)
default_url = "https://us-south.ml.cloud.ibm.com"
entered_url = input(f"IBM Cloud URL [{default_url}]: ").strip()
IBM_CLOUD_URL = entered_url or default_url

# Store them in environment variables (same pattern as your simple watsonx project)
os.environ["IBM_CLOUD_API_KEY"] = IBM_CLOUD_API_KEY
os.environ["IBM_CLOUD_PROJECT_ID"] = IBM_CLOUD_PROJECT_ID
os.environ["IBM_CLOUD_URL"] = IBM_CLOUD_URL

# Also set aliases in case other code expects them
os.environ["WATSONX_APIKEY"] = IBM_CLOUD_API_KEY
os.environ["PROJECT_ID"] = IBM_CLOUD_PROJECT_ID
os.environ["WATSONX_URL"] = IBM_CLOUD_URL

print("\n✅ Credentials stored in environment variables.")

## 3️⃣ watsonx.ai quickstart with `ibm_watsonx_ai`

This section mirrors the quickstart logic from your original project:

- reads credentials from environment variables,
- creates a watsonx.ai `ModelInference` client,
- and generates text from a chosen foundation model.


In [ ]:
import os
from ibm_watsonx_ai import Credentials, APIClient
from ibm_watsonx_ai.foundation_models import ModelInference
from ibm_watsonx_ai.metanames import GenTextParamsMetaNames as GenParams

# Read credentials from environment variables (set in the previous cell)
api_key = os.environ.get("IBM_CLOUD_API_KEY") or os.environ.get("WATSONX_APIKEY")
url = os.environ.get("IBM_CLOUD_URL") or os.environ.get("WATSONX_URL")
project_id = os.environ.get("IBM_CLOUD_PROJECT_ID") or os.environ.get("PROJECT_ID")

if not api_key:
    raise ValueError("Missing API key (IBM_CLOUD_API_KEY or WATSONX_APIKEY).")
if not url:
    raise ValueError("Missing URL (IBM_CLOUD_URL or WATSONX_URL).")
if not project_id:
    raise ValueError("Missing project id (IBM_CLOUD_PROJECT_ID or PROJECT_ID).")

print("✅ Credentials loaded.")
print(f"URL: {url}")
print(f"Project ID: {project_id}")

# Create client & model
credentials = Credentials(url=url, api_key=api_key)
client = APIClient(credentials=credentials, project_id=project_id)  # created for completeness / reuse

# 👉 Choose a model that is available in your watsonx project/space.
# Replace with whatever you actually have access to, for example:
#   - "ibm/granite-13b-chat-v2"
#   - "ibm/granite-8b-code-instruct"
#   - "meta-llama/llama-3-3-70b-instruct"
model_id = "meta-llama/llama-3-3-70b-instruct"  # Change if needed

prompt = "Write a short story about a robot who wants to be a painter."

params = {
    GenParams.DECODING_METHOD: "greedy",
    GenParams.MAX_NEW_TOKENS: 200,
}

model = ModelInference(model_id=model_id, credentials=credentials, project_id=project_id)

print("\n🚀 Sending request to watsonx.ai...")
response = model.generate_text(prompt=prompt, params=params)
print("\n--- watsonx.ai Response ---\n")
print(response)

## 4️⃣ Optional: Use watsonx.ai via LangChain

This section shows how to call watsonx.ai through **LangChain** using
`langchain_ibm.WatsonxLLM`. You can then build chains, tools, and
agents on top of this LLM.


In [ ]:
from dotenv import load_dotenv
from langchain_ibm import WatsonxLLM

# If you prefer using a .env file, you can create it inside the Binder
# environment (using Jupyter's file editor) and this call will load it.
# NOTE: Do NOT commit a .env with real keys to a public repo.
load_dotenv()

api_key = os.environ.get("IBM_CLOUD_API_KEY") or os.environ.get("WATSONX_APIKEY")
url = os.environ.get("IBM_CLOUD_URL") or os.environ.get("WATSONX_URL")
project_id = os.environ.get("IBM_CLOUD_PROJECT_ID") or os.environ.get("PROJECT_ID")

if not api_key or not url or not project_id:
    raise ValueError("Missing credentials for LangChain example.")

# Reuse the same model_id as above or adjust as needed
model_id = "meta-llama/llama-3-3-70b-instruct"  # Change if needed

params = {
    "decoding_method": "greedy",
    "max_new_tokens": 128,
}

llm = WatsonxLLM(
    model_id=model_id,
    url=url,
    apikey=api_key,
    project_id=project_id,
    params=params,
)

print("🚀 Calling watsonx.ai through LangChain...")
response = llm.invoke("Give me 3 concise study tips for learning Python.")
print("\n--- LangChain + watsonx.ai Response ---\n")
print(response)